In [1]:
from intrepid_environment.drone_environment import Controller

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
controller = Controller()

In [4]:
controller.acceleration()

array([-0., -0.,  0.])

In [5]:
controller.linear_velocity()

array([-0., -0.,  0.])

In [6]:
controller.angular_velocity()

array([-0., -0.,  0.])

In [7]:
controller.rotation_angles()

array([-0., -0.,  0.])

In [8]:
controller.control_motors(0.4, 0.7, 0.1, 0.33)

In [10]:
await controller.world.session_step()

In [13]:
controller.control_motors(400, 700, 333, 111)

In [14]:
await controller.world.session_step()

In [ ]:
# nothing happens